## You will learn: how to look which heads ended up dead after pruning.

If you don't know how to load a model yet, look in the notebook __1_Load_model_and_translate__.

### Load trained model

In [1]:
import sys

sys.path.insert(0, 'path_to_the_story_of_heads') # insert your local path to the repo

Load vocabularies.

In [2]:
import pickle
import numpy as np

DATA_PATH = # insert your path
VOC_PATH =  # insert your path

inp_voc = pickle.load(open(VOC_PATH + 'src.voc', 'rb'))
out_voc = pickle.load(open(VOC_PATH + 'dst.voc', 'rb'))

Load model.

In [3]:
%env CUDA_VISIBLE_DEVICES=0

import tensorflow as tf
import lib
import lib.task.seq2seq.models.transformer_head_gates as tr

tf.reset_default_graph()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.99, allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

hp = {
     "num_layers": 6,
     "num_heads": 8,
     "ff_size": 2048,
     "ffn_type": "conv_relu",
     "hid_size": 512,
     "emb_size": 512,
     "res_steps": "nlda", 
    
     "rescale_emb": True,
     "inp_emb_bias": True,
     "normalize_out": True,
     "share_emb": False,
     "replace": 0,
    
     "relu_dropout": 0.1,
     "res_dropout": 0.1,
     "attn_dropout": 0.1,
     "label_smoothing": 0.1,
    
     "translator": "ingraph",
     "beam_size": 4,
     "beam_spread": 3,
     "len_alpha": 0.6,
     "attn_beta": 0,
    
     "concrete_heads": {"enc-self"},
}

model = tr.Model('mod', inp_voc, out_voc, inference_mode='fast', **hp)

env: CUDA_VISIBLE_DEVICES=0


Load trained model parameters.

In [4]:
path_to_ckpt = # insert path to the final checkpoint
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
lib.train.saveload.load(path_to_ckpt, var_list)

## Look which heads are dead.

Gate values are saved in `tf.collection`. To get them, we need to first feed a batch to the model.

Let's create batch for the model. We'll use a couple of sentences about cats, but you can just take a part of your test set.

In [6]:
feed_dict = model.make_feed_dict([('i saw a cat', 'я видел кота'),
                                  ('the cat sat on a mat', 'кот сидел на мате')])

Feed the batch to the model and get gate values from the collection:

In [7]:
tf.get_default_graph().clear_collection("GATEVALUES")
model.transformer.encode(feed_dict['inp'], feed_dict['inp_len'], is_train=False)
gate_values = tf.get_collection("GATEVALUES")

In [8]:
for i in range(model.transformer.num_layers_enc):
    gates = gate_values[i].eval().flatten()
    name = gate_values[i].name.split('/')[1]
    if name.startswith('enc_attn'):
        print("layer = {}, gate values = {}".format(i, gates))

layer = 0, gate values = [0. 0. 1. 1. 0. 0. 0. 1.]
layer = 1, gate values = [1. 0. 0. 1. 0. 1. 1. 0.]
layer = 2, gate values = [0. 0. 0. 0. 1. 0. 1. 0.]
layer = 3, gate values = [1. 0. 1. 0. 1. 1. 0. 0.]
layer = 4, gate values = [0. 1. 1. 0. 1. 0. 1. 0.]
layer = 5, gate values = [1. 0. 0. 0. 0. 0. 0. 1.]


Layers in the model are indexed starting from 0, don't be confused!